In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.contrib.layers as layers
from sklearn.metrics import roc_auc_score

import generate
import plot
import models
import environment

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



# Real data

In [ ]:
# generate fake results
BATCH_SIZE = 500
labels = np.array([1]*BATCH_SIZE + [0]*BATCH_SIZE)
logits = np.random.uniform(size=2*BATCH_SIZE)
    
# make a test plot
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
plot.data1D(ax[0], generate.triangular1D(BATCH_SIZE), generate.triangular1D(BATCH_SIZE))
plot.roc_curve(ax[1], labels, logits)
plt.show()

# Noise distribution

In [ ]:
NOISE_DIM = 3

input_noise = generate.input_noise(NOISE_DIM)
noise = input_noise(300)

fig, ax = plt.subplots(figsize=(5, 5))
_ = ax.scatter(noise[:, 0], noise[:, 1], alpha=0.7)
_ = ax.set_xlabel('noise dimension 1')
_ = ax.set_ylabel('noise dimension 2')
plt.show()

# Make the graphs and run the computation

In [ ]:
COUNTER = 0

In [ ]:

N_PRE_STEPS = 100
N_STEPS = 300
N_DRAW = 10
COUNTER+=1 # so that new graphs are created each time


G = models.GeneratorFullyConnected('Generator_{}'.format(COUNTER), depth=3, n_units=10, output_dim=1)
A = models.AdversaryFullyConnected('Adversary_{}'.format(COUNTER), depth=2, n_units=10, n_classes=2)
env = environment.TFEnvironment(G, A, generate.triangular1D, generate.input_noise(NOISE_DIM))
env.build()
env.initialise_variables()


aurocs = []
adv_losses = []
for i in range(N_PRE_STEPS):
    
    # pretraining step
    env.pretrain_step(BATCH_SIZE)
    
    # get analytics
    to_run = [env.generator.output, env.adversary.labels, env.adversary.logits, env.adversary.loss]
    fake_data, labels, logits, adv_loss = env.run(BATCH_SIZE, *to_run)
    
    # plot only N times
    aurocs.append(roc_auc_score(labels, logits))
    adv_losses.append(adv_loss)
    do_plot = (i in [n* int(N_PRE_STEPS/N_DRAW) for n in range(N_DRAW)]) or (i == N_PRE_STEPS-1)
    
    if False:
        print('Pretraining step {}/{}'.format(i, N_PRE_STEPS))
        fig, ax = plt.subplots(1, 2, figsize=(10, 5))
        plot.data1D(ax[0], generate.triangular1D(BATCH_SIZE), fake_data.ravel())
        plot.roc_curve(ax[1], labels[:,1], logits[:,1])
        plt.show()
        
print('Done with pretraining')

# now run the training steps, and plot the fake and real data
for i in range(N_STEPS):

    # run the computation (discriminator, generator, performance monitoring)
    for _ in range(10):
        env.train_adversary(BATCH_SIZE)
    env.train_generator(BATCH_SIZE)
    
    # get analytics
    to_run = [env.generator.output,
              env.adversary.labels,
              env.adversary.logits,
              env.adversary.loss]   
    fake_data, labels, logits, adv_loss = env.run(BATCH_SIZE, *to_run)
    
    # plot only ten times
    aurocs.append(roc_auc_score(labels, logits))
    adv_losses.append(adv_loss)
    do_plot = (i in [n* int(N_STEPS/N_DRAW) for n in range(N_DRAW)]) or (i == N_STEPS-1)

    if do_plot:
        print('Training step {}/{}'.format(i, N_STEPS))
        fig, ax = plt.subplots(1, 2, figsize=(10, 5))
        plot.data1D(ax[0], generate.triangular1D(BATCH_SIZE), fake_data.ravel())
        plot.roc_curve(ax[1], labels[:,1], logits[:,1])
        plt.show()
    
print('Done with training')

In [ ]:
nplots=2
fig, ax = plt.subplots(1, nplots, sharex=True, figsize=(nplots*6, 5))
ax[0].plot(range(len(aurocs)), aurocs, c='r')
ax[0].set_xlabel('Training steps')
ax[0].set_ylabel('AUROC')
ax[1].plot(range(len(losses_A)), losses_A, c='g')
ax[1].set_xlabel('Training steps')
ax[1].set_ylabel('Adversary loss')
plt.show()

In [ ]:
def f(*args):
    print(args)
    
    l = [*args]
    
    return l

f(1, 2, 3)